In [1]:
%matplotlib inline

import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import randint, uniform
from sklearn import linear_model, preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.linear_model import Lasso, LogisticRegression, Ridge
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_absolute_error, r2_score)
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     train_test_split)
from sklearn.preprocessing import (LabelEncoder, MinMaxScaler, OneHotEncoder,
                                   PolynomialFeatures, RobustScaler,
                                   StandardScaler)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [2]:
# import data
dataset_og = pd.read_csv('Data\Measurements-Transformed')
# kopie maken indien we iets van de originele data nodig hebben
dataset = dataset_og.copy()
dataset.head(5)

,ID,Sex,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
0,371435.0,0.0,21118.0,0.0,-1.75,0.50,55.0,-2.25,1.0,55.0,-1.75,1.00,110.0,-1.25,0.5,110.0
1,371435.0,0.0,20245.0,0.0,-1.75,0.50,65.0,0.00,0.0,0.0,-1.25,0.50,110.0,0.00,0.0,0.0
2,371435.0,0.0,18099.0,0.0,-1.50,0.50,65.0,0.00,0.0,0.0,-1.00,0.50,110.0,0.00,0.0,0.0
3,402916.0,1.0,13825.0,0.0,-3.50,1.50,180.0,-3.50,1.5,180.0,-3.00,1.50,180.0,-3.00,1.5,180.0
4,402916.0,1.0,9653.0,0.0,-2.00,0.75,175.0,0.00,0.0,0.0,-2.00,0.75,180.0,0.00,0.0,0.0


In [3]:
#drop rijen waar < n meting van zijn en houd van de overige de top n meest recente waardes
dataset = dataset.groupby('ID').filter(lambda x: len(x) > 2)
dataset = dataset.groupby('ID').head(3)

Rijen aan elkaar zetten

In [4]:
#3 rijen

from functools import reduce
dfs = [dataset, dataset, dataset]
dataset = reduce(lambda left, right: pd.merge(left, right, on=['ID', 'Sex']), dfs)

dataset = dataset.sort_values(by=['ID', 'Measurement_Age_x'])
dataset = dataset.groupby('ID').head(4)
dataset = dataset.drop_duplicates(subset=['ID', 'Sex'], keep='last')


### Linear regression

In [5]:
# Splitsen in features en targets

y = dataset['Sph-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

In [6]:
# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)


# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=3,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
    
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train))  

[ 3.42984474e-09  6.47544653e-02  1.70172013e-05 -1.80692748e-01
  3.32161801e-01  5.30991576e-02 -1.09607500e-04  7.38686381e-02
  4.71505669e-02  6.62839012e-04  3.41316542e-02 -8.75502867e-02
 -9.75962327e-04 -3.85256766e-02  1.46928727e-01  2.50996735e-04
  3.74065172e-05 -9.16103909e-02  5.67062413e-01  8.68883040e-02
 -8.54175281e-04  2.29389335e-02 -1.04076297e-01  6.71018841e-04
  4.15510360e-02  7.97179118e-02 -8.79624805e-04  3.15814407e-02
 -1.69626101e-01  1.15168454e-03  9.22910162e-06]
r2 score =  0.7419398731823899
dimensie van X_train_poly:  (5376, 528)
dimensie van X_test_poly:  (1345, 528)
R2 score op test set via L2:  0.7032424619092976
R2 score op training set via L2:  0.843308207434877


c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.72061e-26): result may not be accurate.
  overwrite_a=True).T


R2 score op test set via L1:  0.6766911231450107
R2 score op training set via L1:  0.7591147460073867


c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7155.446843041691, tolerance: 5.752361342075889
  positive)


In [8]:
# Splitsen in features en targets

y = dataset['Cyl-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)


# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=3,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
    
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train))  

c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.72061e-26): result may not be accurate.
  overwrite_a=True).T


[-1.59430480e-08  1.43351820e-02 -1.17349365e-05 -5.23170337e-03
 -8.11573561e-03  2.95466913e-01 -3.80524582e-04  3.44770964e-03
  1.16009180e-01  5.41154524e-04  1.60612012e-03 -3.35670538e-02
  2.10581384e-05  3.92996188e-03 -1.06009163e-01 -1.21523842e-04
 -1.79924323e-05 -5.79775670e-02  6.16245633e-03  4.55794030e-01
 -3.94932436e-04 -2.16759593e-02  1.08393401e-01  2.42164018e-04
 -6.96729596e-04  1.59766275e-01 -1.13105168e-04  2.08323775e-02
  1.16713877e-01  3.89701519e-04  3.31584547e-05]
r2 score =  0.5528682939764082
dimensie van X_train_poly:  (5376, 528)
dimensie van X_test_poly:  (1345, 528)
R2 score op test set via L2:  0.4706527922427855
R2 score op training set via L2:  0.7047845995153137
R2 score op test set via L1:  0.49952500983448084
R2 score op training set via L1:  0.5723361365570556


c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 350.5987135591093, tolerance: 0.1613395542689728
  positive)


In [9]:
# Splitsen in features en targets

y = dataset['Sph-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)


# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=3,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
    
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train))  

c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.72061e-26): result may not be accurate.
  overwrite_a=True).T


[ 1.96942365e-08  4.01231074e-02  2.23662899e-05 -1.90463732e-01
  7.36125022e-03  1.28784562e-01 -1.79206291e-03 -9.90706033e-02
  1.11327244e-01  5.39635813e-04  3.35658675e-01 -2.45820667e-01
  6.18038762e-04  1.01553017e-01  3.02267704e-03  1.07830710e-03
  3.58097890e-05 -8.53278695e-02  1.01911827e-01 -1.23277448e-01
 -2.27586795e-04  7.90050894e-02 -2.02398354e-01  1.57526106e-03
  5.30925220e-01  3.68491234e-01 -1.47643510e-03 -5.66956643e-03
 -6.19830454e-02  4.16086900e-04  3.49004220e-06]
r2 score =  0.7434799726541107
dimensie van X_train_poly:  (5376, 528)
dimensie van X_test_poly:  (1345, 528)
R2 score op test set via L2:  0.6987942831174876
R2 score op training set via L2:  0.8287302811421945
R2 score op test set via L1:  0.6827606817406577
R2 score op training set via L1:  0.7374675782003872


c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6987.917973709231, tolerance: 5.15202965843564
  positive)


In [10]:
# Splitsen in features en targets

y = dataset['Cyl-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Linear regression

regmodel = linear_model.LinearRegression()
regmodel.fit(X_train,y_train)
print(regmodel.coef_)
r2 = regmodel.score(X_test,y_test)
print('r2 score = ', r2)

[-3.50934362e-08  1.27789671e-02 -6.08429692e-06 -3.42249342e-03
 -1.84796275e-03 -3.63286328e-02  9.12786813e-05  1.26415702e-02
 -1.10870529e-01  6.00172742e-05 -1.32960181e-02  2.98697734e-01
 -3.44073707e-04 -3.50028039e-03  1.25295333e-01  2.94833653e-04
 -2.39782968e-05 -8.02843634e-02 -9.33848528e-03  1.40511245e-01
 -1.99490535e-04 -3.35319924e-03  1.49151271e-01  2.12852911e-04
  2.24229262e-02  4.85317756e-01 -3.03239046e-04  1.20919342e-02
  7.94696286e-02  3.78763048e-04  3.47209362e-05]
r2 score =  0.5992024316982218


In [11]:
# Modeloptimalisatie en Hyperparameter tuning


# Automatisch toevoegen van hogere orde features

from sklearn.preprocessing import PolynomialFeatures


# Aanmaken van de hogere orde features
graad = 2

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=3,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
    
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train))  

c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=1.72061e-26): result may not be accurate.
  overwrite_a=True).T


dimensie van X_train_poly:  (5376, 528)
dimensie van X_test_poly:  (1345, 528)
R2 score op test set via L2:  0.5960242737777013
R2 score op training set via L2:  0.7128429103036933
R2 score op test set via L1:  0.5744738786494821
R2 score op training set via L1:  0.5935880326814171


c:\users\matth\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 342.4369148581948, tolerance: 0.16492135416666634
  positive)


### Random forest regressor

In [12]:
# Splitsen in features en targets

y = dataset['Sph-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#random forest regressor
RFR_model_sph_far_r = RandomForestRegressor(n_estimators=200)
RFR_model_sph_far_r.fit(X_train,y_train)

RFR_model_sph_far_r.score(X_test,y_test)

0.7829599112982004

In [13]:
# Splitsen in features en targets

y = dataset['Cyl-Far-R'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
RFR_model_cyl_far_r = RandomForestRegressor(n_estimators=200)
RFR_model_cyl_far_r.fit(X_train,y_train)

RFR_model_cyl_far_r.score(X_test,y_test)

0.5802693952509941

In [14]:
# Splitsen in features en targets

y = dataset['Sph-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
RFR_model_sph_far_l = RandomForestRegressor(n_estimators=200)
RFR_model_sph_far_l.fit(X_train,y_train)

RFR_model_sph_far_l.score(X_test,y_test)

0.7751328778701997

In [15]:
# Splitsen in features en targets

y = dataset['Cyl-Far-L'].values
X = dataset.drop(['Add', 'Sph-Far-R', 'Cyl-Far-R', 'Axis-Far-R', 'Sph-Close-R', 'Cyl-Close-R', 'Axis-Close-R', 'Sph-Far-L', 
                  'Cyl-Far-L', 'Axis-Far-L', 'Sph-Close-L', 'Cyl-Close-L', 'Axis-Close-L'],axis=1)

# Splitsen in training set en test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


#random forest regressor
RFR_model_cyl_far_l = RandomForestRegressor(n_estimators=200)
RFR_model_cyl_far_l.fit(X_train,y_train)

RFR_model_cyl_far_l.score(X_test,y_test)

0.6405189902963401

In [16]:
X.head()

,ID,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age_y,Add_y,Sph-Far-R_y,Cyl-Far-R_y,Axis-Far-R_y,Sph-Close-R_y,Cyl-Close-R_y,Axis-Close-R_y,Sph-Far-L_y,Cyl-Far-L_y,Axis-Far-L_y,Sph-Close-L_y,Cyl-Close-L_y,Axis-Close-L_y,Measurement_Age
81129,100063.0,1.0,10897.0,0.0,-2.50,1.0,100.0,0.00,0.0,0.0,-2.5,1.25,85.0,0.00,0.0,0.0,11602.0,0.0,-2.75,1.0,105.0,0.00,0.0,0.0,-3.5,1.75,90.0,0.00,0.0,0.0,13181.0
588,100238.0,1.0,18452.0,0.0,0.00,0.0,0.0,0.75,0.0,0.0,0.0,0.00,0.0,1.75,0.0,0.0,20731.0,0.0,0.00,0.0,0.0,1.50,0.5,75.0,0.0,0.00,0.0,1.75,0.5,105.0,23791.0
101730,100262.0,0.0,16933.0,0.0,0.00,0.0,0.0,1.00,0.0,0.0,0.0,0.00,0.0,1.25,0.0,0.0,17107.0,0.0,0.00,0.0,0.0,1.25,0.0,0.0,0.0,0.00,0.0,1.75,0.0,0.0,19170.0
147549,100320.0,1.0,20588.0,0.0,0.00,0.0,0.0,1.75,0.0,0.0,0.0,0.00,0.0,1.75,0.0,0.0,21477.0,2.0,3.50,0.5,160.0,5.50,0.5,160.0,3.5,0.50,40.0,5.50,0.5,40.0,22573.0
83496,100348.0,0.0,18753.0,0.0,-2.25,1.5,85.0,0.00,0.0,0.0,-1.0,0.50,90.0,0.00,0.0,0.0,18753.0,0.0,-2.25,1.5,85.0,0.00,0.0,0.0,-1.0,0.50,90.0,0.00,0.0,0.0,21449.0


In [17]:
print(dataset.shape)
dataset.head(50)

(6721, 44)


,ID,Sex,Measurement_Age_x,Add_x,Sph-Far-R_x,Cyl-Far-R_x,Axis-Far-R_x,Sph-Close-R_x,Cyl-Close-R_x,Axis-Close-R_x,Sph-Far-L_x,Cyl-Far-L_x,Axis-Far-L_x,Sph-Close-L_x,Cyl-Close-L_x,Axis-Close-L_x,Measurement_Age_y,Add_y,Sph-Far-R_y,Cyl-Far-R_y,Axis-Far-R_y,Sph-Close-R_y,Cyl-Close-R_y,Axis-Close-R_y,Sph-Far-L_y,Cyl-Far-L_y,Axis-Far-L_y,Sph-Close-L_y,Cyl-Close-L_y,Axis-Close-L_y,Measurement_Age,Add,Sph-Far-R,Cyl-Far-R,Axis-Far-R,Sph-Close-R,Cyl-Close-R,Axis-Close-R,Sph-Far-L,Cyl-Far-L,Axis-Far-L,Sph-Close-L,Cyl-Close-L,Axis-Close-L
81129,100063.0,1.0,10897.0,0.00,-2.50,1.00,100.0,0.00,0.00,0.0,-2.50,1.25,85.0,0.00,0.00,0.0,11602.0,0.00,-2.75,1.00,105.0,0.00,0.00,0.0,-3.50,1.75,90.0,0.00,0.00,0.0,13181.0,0.00,-2.75,1.00,105.0,-2.75,1.00,105.0,-3.50,1.75,90.0,-3.50,1.75,90.0
588,100238.0,1.0,18452.0,0.00,0.00,0.00,0.0,0.75,0.00,0.0,0.00,0.00,0.0,1.75,0.00,0.0,20731.0,0.00,0.00,0.00,0.0,1.50,0.50,75.0,0.00,0.00,0.0,1.75,0.50,105.0,23791.0,0.00,2.00,0.50,75.0,2.00,0.50,75.0,2.25,0.50,105.0,2.25,0.50,105.0
101730,100262.0,0.0,16933.0,0.00,0.00,0.00,0.0,1.00,0.00,0.0,0.00,0.00,0.0,1.25,0.00,0.0,17107.0,0.00,0.00,0.00,0.0,1.25,0.00,0.0,0.00,0.00,0.0,1.75,0.00,0.0,19170.0,0.00,1.50,0.00,0.0,1.50,0.00,0.0,2.00,0.00,0.0,2.00,0.00,0.0
147549,100320.0,1.0,20588.0,0.00,0.00,0.00,0.0,1.75,0.00,0.0,0.00,0.00,0.0,1.75,0.00,0.0,21477.0,2.00,3.50,0.50,160.0,5.50,0.50,160.0,3.50,0.50,40.0,5.50,0.50,40.0,22573.0,2.25,4.00,0.25,125.0,6.25,0.25,125.0,4.00,0.25,60.0,6.25,0.25,60.0
83496,100348.0,0.0,18753.0,0.00,-2.25,1.50,85.0,0.00,0.00,0.0,-1.00,0.50,90.0,0.00,0.00,0.0,18753.0,0.00,-2.25,1.50,85.0,0.00,0.00,0.0,-1.00,0.50,90.0,0.00,0.00,0.0,21449.0,0.00,0.75,0.50,85.0,0.75,0.50,85.0,0.00,1.50,90.0,0.00,1.50,90.0
178086,100358.0,1.0,23796.0,0.00,0.75,1.50,0.0,0.00,0.00,0.0,0.50,1.50,0.0,0.00,0.00,0.0,26654.0,0.00,1.00,1.50,180.0,0.00,0.00,0.0,0.75,1.50,180.0,0.00,0.00,0.0,26654.0,0.00,0.00,0.00,0.0,3.25,1.50,180.0,0.00,0.00,0.0,2.75,1.75,180.0
174900,100442.0,0.0,21523.0,2.25,1.50,0.00,0.0,3.75,0.00,0.0,1.50,0.00,0.0,3.75,0.00,0.0,22986.0,2.25,2.00,0.00,0.0,4.25,0.00,0.0,2.00,0.00,0.0,4.25,0.00,0.0,24684.0,2.75,2.00,0.50,90.0,4.75,0.50,90.0,2.00,0.50,180.0,4.75,0.50,180.0
3045,100460.0,0.0,17150.0,1.75,-1.50,1.25,140.0,0.50,1.00,140.0,-2.25,1.00,15.0,-0.75,1.25,15.0,18986.0,2.00,-1.50,1.75,140.0,0.75,1.50,140.0,-2.00,1.50,10.0,-0.25,1.75,10.0,21126.0,2.50,-1.25,1.50,150.0,1.25,1.50,150.0,-1.75,1.50,10.0,0.75,1.50,10.0
120414,100487.0,0.0,28291.0,2.25,0.75,0.00,0.0,3.00,0.00,0.0,0.75,0.00,0.0,3.00,0.00,0.0,29119.0,2.50,1.00,0.50,0.0,3.50,0.00,0.0,1.00,0.00,0.0,3.50,0.50,0.0,30303.0,2.75,0.75,1.00,0.0,3.50,0.75,0.0,1.00,0.75,0.0,3.75,1.00,0.0
173055,100655.0,0.0,24650.0,0.00,-0.75,0.50,5.0,0.00,0.00,0.0,-1.75,1.00,175.0,0.00,0.00,0.0,24650.0,0.00,-0.75,0.50,5.0,0.00,0.00,0.0,-1.75,1.00,175.0,0.00,0.00,0.0,25577.0,0.00,-0.75,0.50,5.0,-0.75,0.50,5.0,-1.75,1.00,175.0,-1.75,1.00,175.0
